# ANALIZA REGRESJI I SZEREGÓW CZASOWYCH PROJEKT
## Paweł Marcinkowski 120660, Wiktor Błaszkiewicz 121314

### Importy

In [1]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pygame
from random import randint, choice, sample

### Stałe zmienne

In [2]:
# Parametry środowiska
GRID_HEIGHT = 20  # liczba wierszy (0 = meta, GRID_HEIGHT-1 = start)
GRID_WIDTH = 20 # liczba kolumn
CELL_SIZE = 30  # rozmiar komórki w pikselach (okno 600×600)
CAR_SPAWN_PROB = 0.2  # prawdopodobieństwo, że w danym wierszu pojawi się samochód
INITIAL_CAR_SPEED = 1 # początkowa prędkość przeszkód
SPEED_INCREMENT = 1  # wzrost prędkości na poziom
SPAWN_INTERVAL = 4  # co ile kroków średnio jedno nowe auto
MID_ROW = GRID_HEIGHT // 2 # środkowy wiersz
LOG_SPAWN_INTERVAL = 2   # co ile kroków średnio nowy pień
LOG_SPEED = 1 # ruch pnia w komórkach na krok
LOG_LENGTH = 3 # długość pnia w komórkach

# Kolory
COLORS = ["red", "orange", "yellow", "green", "blue", "purple"]
COLOR_MAP = {
    "red":    (255,   0,   0),
    "orange": (255, 165,   0),
    "yellow": (255, 255,   0),
    "green":  (  0, 255,   0),
    "blue":   (  0,   0, 255),
    "purple": (128,   0, 128),
}
LOG_COLOR = (139,69,19) 

### Utworzenie środowiska

In [3]:
# Środowisko
class CrossingEnv(gym.Env):
    
    metadata = {"render.modes": ["human", "rgb_array"]}

    def __init__(self):
        super().__init__()
        
        # Obserwacja: macierz GRID_HEIGHT×GRID_WIDTH z wartościami {0,1,2,3}
        # 0 – puste, 1 – gracz, 2 – samochód, 3 - pień lub dziura
        self.observation_space = spaces.Box(
            low=0, high=3,
            shape=(GRID_HEIGHT, GRID_WIDTH),
            dtype=np.int32
        )
        
        # Akcje: 0=up, 1=down, 2=left, 3=right
        self.action_space = spaces.Discrete(4)

        # parametry zmienne w trakcie epizodu
        self.car_speed = INITIAL_CAR_SPEED
        self.score     = 0
    
        # parametry Pygame
        self.window = None
        self.clock  = None
        self.reset()
        self._make_sprites()

    def _make_sprites(self):
        """Tworzy powierzchnie (Surface) z narysowanymi karoseriami aut i żabą."""

        # żaba
        self.player_surf = pygame.Surface((CELL_SIZE, CELL_SIZE), pygame.SRCALPHA)
        pygame.draw.circle(self.player_surf, (0,255,0),
                           (CELL_SIZE//2, CELL_SIZE//2), CELL_SIZE//2 - 2)
        leg_r = 4
        offsets = [(-10,+10),(10,+10),(-10,-10),(10,-10)]
        for dx, dy in offsets:
            pygame.draw.circle(self.player_surf, (0,150,0),
                (CELL_SIZE//2 + dx//2, CELL_SIZE//2 + dy//2), leg_r)

        # auta
        self.car_surfs = {}
        for name, rgb in COLOR_MAP.items():
            surf = pygame.Surface((CELL_SIZE, CELL_SIZE), pygame.SRCALPHA)
            body = pygame.Rect(4, CELL_SIZE//4, CELL_SIZE-8, CELL_SIZE//2)
            pygame.draw.rect(surf, rgb, body, border_radius=4)
            wheel_r = 4
            for wx in (body.left+wheel_r, body.right-wheel_r):
                for wy in (body.top+wheel_r, body.bottom-wheel_r):
                    pygame.draw.circle(surf, (20,20,20), (wx,wy), wheel_r)
            self.car_surfs[name] = surf
            
    # funkcja resetująca
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        
        # czysta plansza
        self.grid = np.zeros((GRID_HEIGHT, GRID_WIDTH), dtype=np.int32)
        
        # gracz: dolny środek
        self.player_pos = [GRID_HEIGHT - 1, GRID_WIDTH // 2]
        self.grid[self.player_pos[0], self.player_pos[1]] = 1

        # Wybieramy dwie całkowicie wodne linie
        candidates = list(range(1, MID_ROW))
        self.water_rows = sample(candidates, 2)

        # Dziury (pojedyncze pola wody) w pozostałych wierszach górnej połowy
        self.holes = []
        for row in candidates:
            if row not in self.water_rows:
                for _ in range(3):
                    col = randint(0, GRID_WIDTH-1)
                    self.holes.append({'row': row, 'col': col})

        # Pnie tylko w pełnych wodnych wierszach
        self.logs = []
        # Samochody
        self.cars = []
        # Ukończenie epizodu
        self.done = False

        return self._get_obs(), {}

    def _get_obs(self):
        obs = self.grid.copy()

        # Przypisujemy odpowiednie rodzaje pól do obiektów
        
        # samochody = 2
        for c in self.cars:
            r, c0 = c['row'], c['col']
            if 0 <= r < GRID_HEIGHT and 0 <= c0 < GRID_WIDTH:
                obs[r, c0] = 2

        # pnie = 3
        for log in self.logs:
            r, c0 = log['row'], log['col']
            for x in range(log['length']):
                cc = c0 + x
                if 0 <= r < GRID_HEIGHT and 0 <= cc < GRID_WIDTH:
                    obs[r, cc] = 3

        # dziury = 3
        for h in self.holes:
            r, c0 = h['row'], h['col']
            if 0 <= r < GRID_HEIGHT and 0 <= c0 < GRID_WIDTH:
                obs[r, c0] = 3

        return obs

    # krok danego epizodu    
    def step(self, action):
        if self.done:
            return self._get_obs(), 0, True, False, {}

        # ruch gracza
        self.grid[self.player_pos[0], self.player_pos[1]] = 0
        if   action == 0 and self.player_pos[0] > 0:
            self.player_pos[0] -= 1
        elif action == 1 and self.player_pos[0] < GRID_HEIGHT-1:
            self.player_pos[0] += 1
        elif action == 2 and self.player_pos[1] > 0:
            self.player_pos[1] -= 1
        elif action == 3 and self.player_pos[1] < GRID_WIDTH-1:
            self.player_pos[1] += 1
        self.grid[self.player_pos[0], self.player_pos[1]] = 1

        reward = -1

        # przesuwanie samochodów
        new_cars = []
        for car in self.cars:
            car['col'] -= self.car_speed
            if car['col'] >= 0:
                new_cars.append(car)
        self.cars = new_cars

        # generowanie samochodów
        if randint(1, SPAWN_INTERVAL)==1:
            row = randint(MID_ROW, GRID_HEIGHT-2)
            self.cars.append({'row':row,'col':GRID_WIDTH-1,'color':choice(COLORS)})

        # przesuwanie pni tylko w wierszach wypełnionych wodą
        new_logs = []
        for log in self.logs:
            log['col'] += log['speed']
            if log['col'] < GRID_WIDTH:
                new_logs.append(log)
        self.logs = new_logs

        # generowanie pni tylko w wierszach wypełnionych wodą
        if randint(1, LOG_SPAWN_INTERVAL)==1:
            row = choice(self.water_rows)
            self.logs.append({
                'row':    row,
                'col':    -LOG_LENGTH,
                'length': LOG_LENGTH,
                'speed':  LOG_SPEED
            })

        # obsługa strefy całkowicie wodnej
        pr, pc = self.player_pos
        if pr in self.water_rows:
            
            # sprawdzamy czy gracz jest na pniu
            is_on_log = any(
                log['row']==pr and log['col']<=pc<log['col']+log['length']
                for log in self.logs
            )
            
            if is_on_log:
                for log in self.logs:
                    # przesuwamy gracza wraz pniem
                    if log['row']==pr and log['col']<=pc<log['col']+log['length']:
                        self.player_pos[1] += log['speed']
                        break
                if not (0<=self.player_pos[1]<GRID_WIDTH):
                    reward = -100; self.done = True
                    
            # jeśli nie oznacza to, że jest w wodzie - koniec gry
            else:
                reward = -100; self.done = True

        elif 1 <= pr < MID_ROW:
            # trawa z dziurami wodnymi
            # wpadek do dziury - koniec gry
            if any(h['row']==pr and h['col']==pc for h in self.holes):
                reward = -100; self.done = True

        # kolizja z autem
        if any(car['row']==self.player_pos[0] and car['col']==self.player_pos[1]
               for car in self.cars):
            reward = -100; self.done = True

        # meta
        if self.player_pos[0]==0 and not self.done:
            reward = 100
            self.score += 1
            self.car_speed += SPEED_INCREMENT
            self.player_pos = [GRID_HEIGHT-1, GRID_WIDTH//2]
            self.cars = []

        # aktualizacja siatki i zwrot wszystkich zmiennych
        self.grid = np.zeros((GRID_HEIGHT, GRID_WIDTH), dtype=np.int32)
        self.grid[self.player_pos[0], self.player_pos[1]] = 1
        return self._get_obs(), reward, self.done, False, {}

    # renderowanie reprezentacji graficznej
    def render(self, mode="human"):
        if self.window is None:
            pygame.init()
            size = (GRID_WIDTH*CELL_SIZE, GRID_HEIGHT*CELL_SIZE)
            self.window = pygame.display.set_mode(size)
            pygame.display.set_caption("Gym Frogger")
            self.clock = pygame.time.Clock()
            self._make_sprites()

        # Dolna trawa i droga / górna meta-trawa
        self.window.fill((0,0,128))
        bottom = pygame.Rect(0,(GRID_HEIGHT-1)*CELL_SIZE,GRID_WIDTH*CELL_SIZE,CELL_SIZE)
        top    = pygame.Rect(0,0,                 GRID_WIDTH*CELL_SIZE,CELL_SIZE)
        pygame.draw.rect(self.window,(0,200,0),bottom)
        pygame.draw.rect(self.window,(0,200,0),top)
        road = pygame.Rect(0,MID_ROW*CELL_SIZE,GRID_WIDTH*CELL_SIZE,(GRID_HEIGHT-1-MID_ROW)*CELL_SIZE)
        pygame.draw.rect(self.window,(50,50,50),road)

        # Górna połowa: woda i trawa z dziurami
        for row in range(1, MID_ROW):
            for col in range(GRID_WIDTH):
                cell = pygame.Rect(col*CELL_SIZE, row*CELL_SIZE, CELL_SIZE, CELL_SIZE)
                if row in self.water_rows:
                    pygame.draw.rect(self.window,(0,0,128),cell)
                else:
                    pygame.draw.rect(self.window,(0,200,0),cell)
                    if any(h['row']==row and h['col']==col for h in self.holes):
                        pygame.draw.rect(self.window,(0,0,128),cell)

        # Siatka
        for row in range(GRID_HEIGHT):
            for col in range(GRID_WIDTH):
                r = pygame.Rect(col*CELL_SIZE, row*CELL_SIZE, CELL_SIZE, CELL_SIZE)
                pygame.draw.rect(self.window,(50,50,50),r,1)

        # Pnie
        for log in self.logs:
            r, c0 = log['row'], log['col']
            rect = pygame.Rect(c0*CELL_SIZE, r*CELL_SIZE, log['length']*CELL_SIZE, CELL_SIZE)
            pygame.draw.rect(self.window, LOG_COLOR, rect)

        # Samochody
        for car in self.cars:
            r, c0, color = car['row'], car['col'], car['color']
            self.window.blit(self.car_surfs[color], (c0*CELL_SIZE, r*CELL_SIZE))

        # Żaba
        pr, pc = self.player_pos
        self.window.blit(self.player_surf, (pc*CELL_SIZE, pr*CELL_SIZE))

        pygame.display.flip()
        self.clock.tick(10)

    def _get_rgb_array(self):
        return pygame.surfarray.array3d(pygame.display.get_surface()).transpose(1,0,2)

    def close(self):
        if self.window:
            pygame.quit()
            self.window = None

### Demonstracja gry przed jakimkolwiek treningiem modelu

In [7]:
if __name__ == "__main__":
    env = CrossingEnv()
    obs, _ = env.reset()
    total_reward = 0
    done = False

    env.render()
    
    while not done:
        # obsługa Pygame events, żeby okno reagowało na zamknięcie
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True

        action = env.action_space.sample()
        obs, reward, done, truncated, info = env.step(action)
        total_reward += reward
        env.render()

    print("Episode finished. Total reward:", total_reward)
    env.close()

Episode finished. Total reward: -133


### Manualne testowanie gry

In [8]:
if __name__ == "__main__":
    env = CrossingEnv()
    obs, _ = env.reset()

    total_reward = 0
    done = False

    # narysuj pierwszy raz, żeby pygame.init() i okno były gotowe
    env.render()

    action = None
    clock = pygame.time.Clock()

    while not done:
        # zbieranie eventów – tylko do ustawienia action lub zamknięcia okna
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True

            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    action = 0
                elif event.key == pygame.K_DOWN:
                    action = 1
                elif event.key == pygame.K_LEFT:
                    action = 2
                elif event.key == pygame.K_RIGHT:
                    action = 3

        # Na każdą klatkę robimy krok w środowisku
        # action może być None (żadna strzałka), wtedy gracz się nie ruszy
        obs, reward, done, truncated, info = env.step(action)
        total_reward += reward

        # Rysujemy stan po kroku
        env.render()

        # Resetujemy action, żeby kolejny krok był „czysty”
        action = None

        # Zachowaj stałe FPS (tu 10)
        clock.tick(10)

    print("Episode finished. Total reward:", total_reward)
    env.close()

IndexError: index 20 is out of bounds for axis 1 with size 20